# Klassifikationshierarchie

In [1]:
import pandas as pd
import numpy as np

headers_wz = ['id', 'code', 'definition', 'isic']
wz = pd.read_csv('../data/raw/klassifikationenWZ2008.csv', sep=';', names=headers_wz, encoding='latin-1')

wz.drop('isic', axis=1, inplace=True)
wz.set_index('id', inplace=True)

wz.head()

,code,definition
id,,
1,A,"ABSCHNITT A - LAND- UND FORSTWIRTSCHAFT, FISCH..."
2,01,"Landwirtschaft, Jagd und damit verbundene Täti..."
3,01.1,Anbau einjähriger Pflanzen
4,01.11,"Anbau von Getreide (ohne Reis), Hülsenfrüchten..."
5,01.11.0,"Anbau von Getreide (ohne Reis), Hülsenfrüchten..."


In [2]:
uppercase_mask = wz['code'].str.isupper()
sections = wz[uppercase_mask]

sections["definition"] = sections["definition"].str[14:]

sections.columns=['section', 'section_name']

sections.set_index('section', inplace=True)

sections

C:\Users\Ray\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,section_name
section,
A,"LAND- UND FORSTWIRTSCHAFT, FISCHEREI"
B,BERGBAU UND GEWINNUNG VON STEINEN UND ERDEN
C,VERARBEITENDES GEWERBE
D,ENERGIEVERSORGUNG
E,WASSERVERSORGUNG; ABWASSER- UND ABFALLENTSORGU...
F,BAUGEWERBE
G,HANDEL; INSTANDHALTUNG UND REPARATUR VON KRAFT...
H,VERKEHR UND LAGEREI
I,GASTGEWERBE


In [3]:
subsections = wz[~uppercase_mask]

first_subsection_mask = subsections['code'].str.len() < 3
subs_1 = subsections[first_subsection_mask]

subs_1.columns=['sub_1', 'sub_1_name']
subs_1.set_index('sub_1', inplace=True)

subs_1.head()

,sub_1_name
sub_1,
01,"Landwirtschaft, Jagd und damit verbundene Täti..."
02,Forstwirtschaft und Holzeinschlag
03,Fischerei und Aquakultur
05,Kohlenbergbau
06,Gewinnung von Erdöl und Erdgas


In [4]:
second_subsection_mask = (subsections['code'].str.len() > 3) & (subsections['code'].str.len() <=5)
subs_2 = subsections[second_subsection_mask]

subs_2['code_1'] = subs_2['code'].str[:2]
subs_2['code_2'] = subs_2['code'].str[3:]

subs_2.columns=['code', 'sub_2_name', 'sub_1', 'sub_2']

subs_2.drop('code', axis=1, inplace=True)
subs_2.set_index(['sub_1', 'sub_2'], inplace=True)

subs_2.head()

C:\Users\Ray\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ray\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ray\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


sub_2_name
sub_1 sub_2                                                   
01    1                             Anbau einjähriger Pflanzen
      11     Anbau von Getreide (ohne Reis), Hülsenfrüchten...
      12                                        Anbau von Reis
      13     Anbau von Gemüse und Melonen sowie Wurzeln und...
      14                                  Anbau von Zuckerrohr

# Firmen

In [5]:
headers_dom = ['url', 'section', 'code']
dom = pd.read_csv('../data/raw/20181018 Domain+WZ2008 Hackathon INOBAS.csv', sep=';', names=headers_dom, encoding='latin-1', skiprows=1)

dom.head()

,url,section,code
0,www.hoberg-driesch.de,L,68.31
1,www.hgdf.de,K,64.20
2,www.gasag.de,D,35.30
3,www.ergodirekt.de,K,65.11
4,www.wahler.de,M,70.10


In [6]:
split =  pd.DataFrame(dom['code'].str.split('.').tolist(),
                                   columns = ['sub_1','sub_2'])
dom_split = pd.concat([dom, split], axis=1)
dom_split.drop('code', axis=1, inplace=True)
dom_split.head()

,url,section,sub_1,sub_2
0,www.hoberg-driesch.de,L,68,31
1,www.hgdf.de,K,64,20
2,www.gasag.de,D,35,30
3,www.ergodirekt.de,K,65,11
4,www.wahler.de,M,70,10


In [7]:
dom_s = dom_split.merge(sections, on='section')
dom_s.head()

,url,section,sub_1,sub_2,section_name
0,www.hoberg-driesch.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN
1,www.muenchner-bank.de,L,68,10,GRUNDSTÜCKS- UND WOHNUNGSWESEN
2,www.stadt-willich.de,L,68,10,GRUNDSTÜCKS- UND WOHNUNGSWESEN
3,www.treureal.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN
4,www.sparkasse-karlsruhe-ettlingen.de,L,68,10,GRUNDSTÜCKS- UND WOHNUNGSWESEN


In [8]:
dom_1 = dom_s.merge(subs_1, on='sub_1')
dom_1.head()

,url,section,sub_1,sub_2,section_name,sub_1_name
0,www.hoberg-driesch.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen
1,www.muenchner-bank.de,L,68,10,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen
2,www.stadt-willich.de,L,68,10,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen
3,www.treureal.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen
4,www.sparkasse-karlsruhe-ettlingen.de,L,68,10,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen


In [9]:
dom_2 = dom_1.merge(subs_2, on=['sub_1','sub_2'])
dom_2.head()

,url,section,sub_1,sub_2,section_name,sub_1_name,sub_2_name
0,www.hoberg-driesch.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh..."
1,www.treureal.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh..."
2,www.immo-ostalb.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh..."
3,www.trauco.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh..."
4,www.kskgrossgerau.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh..."


In [10]:
import pickle

with open('../data/processed/companies_enchanced_labeling.pickle', 'wb') as f:
    pickle.dump(dom_2, f)

In [11]:
section_sizes = dom_2.groupby(['section','section_name']).size().to_frame('size').sort_values(by=['size'])

section_sizes

,,size
section,section_name,
A,"LAND- UND FORSTWIRTSCHAFT, FISCHEREI",8
B,BERGBAU UND GEWINNUNG VON STEINEN UND ERDEN,8
O,"ÖFFENTLICHE VERWALTUNG, VERTEIDIGUNG; SOZIALVERSICHERUNG",39
R,"KUNST, UNTERHALTUNG UND ERHOLUNG",48
E,WASSERVERSORGUNG; ABWASSER- UND ABFALLENTSORGUNG UND BESEITIGUNG VON UMWELTVERSCHMUTZUNGEN,91
I,GASTGEWERBE,92
D,ENERGIEVERSORGUNG,93
P,ERZIEHUNG UND UNTERRICHT,111
L,GRUNDSTÜCKS- UND WOHNUNGSWESEN,167


Wenn wir die 4 Kategorien mit unter 90 Datenpunkten ausschließen, sind wir bei 99% der Daten 